In [94]:
# import module
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime
from pandas import Series,DataFrame
from math import *
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.linear_model import OLS
import statsmodels.formula.api as sm
import time
import random
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgunbd.ttf').get_name()
mpl.rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

## sklearn
from sklearn import linear_model
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import (GradientBoostingRegressor, GradientBoostingClassifier, RandomForestRegressor, VotingClassifier, VotingRegressor)
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import *
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

#LightGBM
from lightgbm import LGBMClassifier, LGBMRegressor


# xgboost
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance
import xgboost as xgb

# randomforest
from sklearn.ensemble import RandomForestClassifier

# MLP
from sklearn.neural_network import MLPClassifier


In [53]:
# load data
df = pd.read_csv('D:/data/row_sihwa.csv',encoding='CP949')
china = pd.read_csv('D:/data/chinapm_final.csv',encoding='CP949')

In [54]:
# preprocessing
df.drop('Unnamed: 0',axis=1,inplace=True)
df = df[['site', 'date', 'so2', 'co', 'o3', 'no2', 'pm10', 'pm25', 'weather',
       'wind_speed', 'wind_direction', 'rain_yn', 'humid']]
df = df.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12]]
df.loc[df['weather'] <= -30, 'weather' ] = np.float("nan")
df.isnull().sum()

site                 0
date                 0
so2                566
co                 487
o3                 448
no2                439
pm10               672
pm25              8002
weather            403
wind_speed         351
wind_direction     351
rain_yn            384
humid              333
dtype: int64

In [55]:
## 파생변수 ymd, md, month, season ,weekday,
def date_split(df) :
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M')
    df['ymd'] = [d.date() for d in df['date']]
    df['month'] = [d.month for d in df['date']]
    df['time'] = [d.time() for d in df['date']]
    df['time'] = df['time'].apply(lambda x: x.strftime('%H'))
    df['md'] = df['ymd'].apply(lambda x: x.strftime('%m-%d'))
    df['weekday'] =df['date'].apply(lambda x: x.strftime('%A'))
def season(x):
    if '05-31'>= x>='03-01':
        return('봄')
    elif '11-30' >= x >= '09-01':
        return('가을')
    elif '08-31' >=x >= '06-01':
        return('여름')
    else: return('겨울')

In [56]:
# modeling Definition
def average_modeling(models) :
    for i,m in enumerate(models):
        print(i, m.__class__)
        m.fit(x_train, y_train)

    models = sorted(models, key=lambda m: mean_squared_error(y_test, m.predict(x_test)))


    y_preds = np.array([m.predict(x_test) for m in models]).T
    y_preds_mean = y_preds.mean(axis=1)
    df_predict = pd.DataFrame({'Actual': y_test, 'Predicted': y_preds_mean})
    print("RMSE : {:.3f}".format((mean_squared_error(y_test, y_preds_mean))**0.5))

    y_pred_log = np.array([m.predict(x_test) for m in models]).T.dot(
    np.linspace(1.0, 0.0, len(models))/sum(np.linspace(1.0, 0.0, len(models))))
    df_predict = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_log})
    print("RMSE : {:.3f}".format((mean_squared_error(y_test, y_pred_log))**0.5))

In [57]:
date_split(df)
df = df.dropna()
df['season'] = df['md'].apply(season)
df = df[df['site']=='시화산단']

In [58]:
df.drop(['site','md','ymd'],axis=1,inplace=True)
china.drop(['Unnamed: 0'],axis=1, inplace=True)

In [59]:
df_column = df.columns.tolist()
df_column.remove('date')
df_column.extend(['PM2.5_24h','PM10','PM10_24h','PM2.5'])

In [60]:
df['date'] = df['date'].astype(str)
df_china = pd.merge(df,china, on=['date'],how='left')
df_china.dropna(inplace=True)
df_china.drop(['date'],axis=1,inplace=True)
df_china = df_china[df_column]

In [64]:
# set china pm2.5_24h, pm10_24h
df_china.drop(['PM10','PM2.5'],axis=1,inplace=True)
data = df_china.copy()
data['pm25_window_24'] = data['pm25'].rolling(window=6).mean()
data.dropna(inplace=True)
random.seed(100)

## x, y setting
X = data.drop(["pm25",'pm10'], axis=1)
Y = data["pm25"]

# one-hot encoding
# object 변수만 해당
for i in ["season","weekday",'time']:
    X[i] = pd.get_dummies(X[i])
# trian / test split 0.8/ 0.2
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [65]:
# hyper parameter tunning
models = []
models = [RandomForestRegressor(n_estimators=n, random_state =5) for n in [ 30, 50, 100, 200]]
models+=[GradientBoostingRegressor(random_state =5)]
models+=[KernelRidge(alpha=0.6, kernel='polynomial', degree=i, coef0=2.5) for i in range(3,5)]
models +=[XGBRegressor(max_depth= 8,
                       objective='reg:squarederror',
                       min_child_weight=1.111, n_estimator= n, subsample=0.6, random_state =5) for n in [30,50,100,200,500,1000]]

average_modeling(models)

0 <class 'sklearn.ensemble.forest.RandomForestRegressor'>
1 <class 'sklearn.ensemble.forest.RandomForestRegressor'>
2 <class 'sklearn.ensemble.forest.RandomForestRegressor'>
3 <class 'sklearn.ensemble.forest.RandomForestRegressor'>
4 <class 'sklearn.ensemble.gradient_boosting.GradientBoostingRegressor'>
5 <class 'sklearn.kernel_ridge.KernelRidge'>
6 <class 'sklearn.kernel_ridge.KernelRidge'>
7 <class 'xgboost.sklearn.XGBRegressor'>
8 <class 'xgboost.sklearn.XGBRegressor'>
9 <class 'xgboost.sklearn.XGBRegressor'>
10 <class 'xgboost.sklearn.XGBRegressor'>
11 <class 'xgboost.sklearn.XGBRegressor'>
12 <class 'xgboost.sklearn.XGBRegressor'>
RMSE : 8.016
RMSE : 8.028


# Categorical

In [113]:
df = pd.read_csv('D:/data/row_sihwa.csv',encoding='CP949')
china = pd.read_csv('D:/data/chinapm_final.csv',encoding='CP949')

In [114]:
# preprocessing
df.drop('Unnamed: 0',axis=1,inplace=True)
df = df[['site', 'date', 'so2', 'co', 'o3', 'no2', 'pm10', 'pm25', 'weather',
       'wind_speed', 'wind_direction', 'rain_yn', 'humid']]
df = df.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12]]
df.loc[df['weather'] <= -30, 'weather' ] = np.float("nan")
df.isnull().sum()

site                 0
date                 0
so2                566
co                 487
o3                 448
no2                439
pm10               672
pm25              8002
weather            403
wind_speed         351
wind_direction     351
rain_yn            384
humid              333
dtype: int64

In [115]:
## EDA 위한 파생변수 ymd, md, month, season ,weekday, pm categori 추가
def date_split(df) :
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M')
    df['ymd'] = [d.date() for d in df['date']]
    df['month'] = [d.month for d in df['date']]
    df['time'] = [d.time() for d in df['date']]
    df['time'] = df['time'].apply(lambda x: x.strftime('%H'))
    df['md'] = df['ymd'].apply(lambda x: x.strftime('%m-%d'))
    df['weekday'] =df['date'].apply(lambda x: x.strftime('%A'))
def season(x):
    if '05-31'>= x>='03-01':
        return('봄')
    elif '11-30' >= x >= '09-01':
        return('가을')
    elif '08-31' >=x >= '06-01':
        return('여름')
    else: return('겨울')

def categorical_PM(x):
    if 8>= x>=0:
        return('0')
    elif 15 >= x >= 9 :
        return('1')
    elif 20 >=x >= 16:
        return('2')
    elif 25 >=x >= 21:
        return('3')
    elif 37 >=x >= 26:
        return('4')
    elif 50 >=x >= 38:
        return('5')
    elif 75 >=x >= 51:
        return('6')
    else: return('7') 
    
# categorical modeling definition

def categorical_modeling(models):
    votingC = VotingClassifier(estimators=[(f'{m}' , m) for m in models], voting='hard')
 
    votingC = votingC.fit(x_train, y_train)

    #예측 진행
    y_pred = votingC.predict(x_test) 

    # ACC, F1-score check
    print("ACC : ", accuracy_score(y_test, y_pred))
    print("f1-score : ", f1_score(y_test, y_pred, average='macro'))

In [116]:
date_split(df)
df = df.dropna()
df['season'] = df['md'].apply(season)
df['PM_categorical'] = df['pm25'].apply(categorical_PM)
df['PM_categorical'] = df['PM_categorical'].astype('category')
df = df[df['site']=='시화산단']
df.drop(['site','md','ymd'],axis=1,inplace=True)
df = df.reset_index(drop=True)
china.drop(['Unnamed: 0'],axis=1, inplace=True)

In [117]:
df_column = df.columns.tolist()
df_column.remove('date')
df_column.extend(['PM2.5_24h','PM10','PM10_24h','PM2.5'])

In [118]:
df['date'] = df['date'].astype(str)
df_china = pd.merge(df,china, on=['date'],how='left')
df_china.dropna(inplace=True)
df_china.drop(['date'],axis=1,inplace=True)

In [119]:
df_china.drop(['PM10','PM2.5'],axis=1,inplace=True)
data = df_china.copy()
data['pm25_window_24'] = data['pm25'].rolling(window=6).mean()
data['pm10_window_24'] = data['pm10'].rolling(window=6).mean()
data.dropna(inplace=True)
random.seed(100)

## x, y setting
X = data.drop(["pm25",'pm10','PM_categorical'], axis=1)
Y = data["PM_categorical"]

# one-hot encoding
# object 변수만 해당
for i in ["season", "weekday",'time']:
    X[i] = pd.get_dummies(X[i])
    
# trian / test split 0.8/ 0.2
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0,stratify=Y)

In [120]:
# add lightGBM
models = [RandomForestClassifier(n_estimators=n,random_state =5) for n in [50, 100, 200, 600]]
models+=[GradientBoostingClassifier(n_estimators=n_e, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   random_state =5) for n_e in [3000, 5000]]
models+=[XGBClassifier(max_depth=12,n_estimators=n, random_state =5) for n in [50, 100,200,400,1000]]
models += [LGBMClassifier( n_estimators=n, max_depth=8, reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            random_state=5)for n in [50,100,500,1000]]

categorical_modeling(models)

ACC :  0.6004291845493562
f1-score :  0.6108693775541216


In [ ]:
cv = StratifiedKFold(5, shuffle=True, random_state=5)
cross_val_score(categorical_modeling(models), X, Y, scoring="accuracy", cv=cv)

In [106]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

le = LabelEncoder()
y_train = le.fit_transform(y_train)
x_data = np.vstack([x_train,x_test])
mas = MaxAbsScaler()
n_x_data = mas.fit_transform(x_data)
train_length = len(x_train)
x_test = n_x_data[train_length :,:]
x_train = n_x_data[0:train_length,:]

In [108]:
seed=5
models = [
            'RFC',
            'GBC',
            'XGB',
            'LGBM'
            ]
clfs = [RandomForestClassifier(n_estimators=n,random_state =5) for n in [50, 100, 200, 600]]
clfs+=[GradientBoostingClassifier(n_estimators=n_e, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   random_state =5) for n_e in [3000, 5000]]
clfs+=[XGBClassifier(max_depth=12,n_estimators=n, random_state =5) for n in [50, 100,200,400,1000]]
clfs += [LGBMClassifier( n_estimators=n, max_depth=8, reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            random_state=5)for n in [50,100,500,1000]]

In [109]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold

pred_train_models = []
pred_test_models = []

kfold = 3 # use a bigger number

cv = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=seed)
cvfolds = list(cv.split(x_train,y_train))

for j,clf in enumerate(clfs):
    print(j,clf)
    dataset_test_j = 0 
    dataset_train_j = np.zeros((x_train.shape[0],len(np.unique(y_train))))
    for i,(train_index, test_index) in enumerate(cvfolds):
        n_x_train, n_x_val = x_train[train_index], x_train[test_index]
        n_y_train, n_y_val = y_train[train_index], y_train[test_index]
        print('fold ' + str(i))        
        clf.fit(n_x_train,n_y_train)
        dataset_train_j[test_index,:] = clf.predict_proba(n_x_val)
        dataset_test_j += clf.predict_proba(x_test)
    pred_train_models.append(dataset_train_j)
    pred_test_models.append(dataset_test_j/float(kfold))
    
pred_blend_train = np.hstack(pred_train_models)
pred_blend_test = np.hstack(pred_test_models)

0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=5, verbose=0,
                       warm_start=False)
fold 0
fold 1
fold 2
1 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=5, verbose=0,
                       warm_start=False)


In [110]:
print('\Blending results with a Logistic Regression ... ')

blendParams = {'C':[1000],'tol':[0.01]} # test more values in your local machine
clf = GridSearchCV(LogisticRegression(solver='newton-cg', multi_class='multinomial'), blendParams, scoring='accuracy',
                   refit='True', n_jobs=-1, cv=5)
clf.fit(pred_blend_train,y_train)
print('The Best parameters of the blending model\n{}'.format(clf.best_params_))
print('The best score:{}'.format(clf.best_score_))

\Blending results with a Logistic Regression ... 
The Best parameters of the blending model
{'C': 1000, 'tol': 0.01}
The best score:0.5826357587465121


# Add China PM10, PM2.5 3 days after